In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context="notebook", style="white", palette=sns.color_palette("RdBu"), color_codes=False)

# load data

943 users, 1682 movies.

X is the features of movie.

Theta is the parameters of user.

Y is a num_movies $\times$ num_users matrix, stores the ratings $y^{(i,j)}$ from 1 to 5.

R is an binary-valued indicator matrix, R(i,j)=1 if user j gave a rating to movie i, and R(i,j)=0 otherwise.

In [2]:
movies_mat = sio.loadmat('./data/ex8_movies.mat')
movies_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Y', 'R'])

In [3]:
Y, R = movies_mat.get('Y'), movies_mat.get('R')
Y.shape, R.shape

((1682, 943), (1682, 943))

In [4]:
param_mat = sio.loadmat('./data/ex8_movieParams.mat')
param_mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'X', 'Theta', 'num_users', 'num_movies', 'num_features'])

In [5]:
X, theta = param_mat.get('X'), param_mat.get('Theta')
X.shape, theta.shape

((1682, 10), (943, 10))

In [6]:
m, u = Y.shape
n = X.shape[1]
m, u, n

(1682, 943, 10)

In [7]:
movie_list = []

with open('./data/movie_ids.txt', encoding='latin-1') as f:
    for line in f:
        tokens = line.strip().split(' ')
        movie_list.append(' '.join(tokens[1:]))

len(movie_list)        

1682

# cost function

In [8]:
def serialize(X, theta):
    return np.concatenate((X.ravel(), theta.ravel()))

def deserialize(param, n_movie, n_user, n_feature):
    return param[:n_movie*n_feature].reshape(n_movie, n_feature), param[n_movie*n_feature:].reshape(n_user, n_feature)

![](img/cost.png)

In [9]:
def cost(param, Y, R, n_feature):
    '''
    X(movie, feature), (1682, 10)
    theta(user, feature), (943, 10)
    '''
    n_movie, n_user = Y.shape
    X, theta = deserialize(param, n_movie, n_user, n_feature)
    inner = (X @ theta.T - Y) * R
    return (inner ** 2).sum() / 2 

![](img/regularized_cost.png)

In [10]:
def regularized_cost(param, Y, R, n_feature, l=1):
    reg_term = (param ** 2).sum() / (l * 2)
    return cost(param, Y, R, n_feature) + reg_term

# gradient

![](img/gradient.png)

In [11]:
def gradient(param, Y, R, n_feature):
    n_movie, n_user = Y.shape
    X, theta = deserialize(param, n_movie, n_user, n_feature)
    '''
    inner: n_movie * n_user
    X: n_movie * n_feature
    theta: n_user * n_feature
    '''
    inner = (X @ theta.T - Y) * R
    X_grad = inner @ theta
    theta_grad = inner.T @ X 
    
    return serialize(X_grad, theta_grad)

![](img/regularized_gradient.png)

In [12]:
def regularized_gradient(param, Y, R, n_feature, l=1):
    grad = gradient(param, Y, R, n_feature)
    reg_term = l * param
    
    return grad + reg_term

In [13]:
param = serialize(X, theta)
cost(param, Y, R, 10), regularized_cost(param, Y, R, 10, l=1)

(27918.64012454421, 32520.682450229557)

## add user

In [14]:
ratings = np.zeros(1682)
ratings[0] = 4
ratings[11] = 4
ratings[21] = 1
ratings[30] = 5
ratings[35] = 4
ratings[60] = 3
ratings[76] = 4
ratings[90] = 2
ratings[110] = 5
ratings[330] = 4

In [15]:
Y, R = movies_mat.get('Y'), movies_mat.get('R')
Y = np.insert(Y, 0, ratings, axis=1)
R = np.insert(R, 0, ratings != 0, axis=1)
Y.shape, R.shape

((1682, 944), (1682, 944))

In [16]:
n_feature = 50
n_movie, n_user = Y.shape
l = 10

X = np.random.standard_normal((n_movie, n_feature))
theta = np.random.standard_normal((n_user, n_feature))
X.shape, theta.shape

((1682, 50), (944, 50))

In [17]:
param = serialize(X, theta)

Y_norm = Y - Y.mean()
Y.mean(), Y_norm.mean()

(0.22233292690300085, 7.516195780488978e-17)

# training

In [18]:
import scipy.optimize as opt

In [19]:
res = opt.minimize(fun=regularized_cost, x0=param, args=(Y_norm, R, n_feature, l),
                  method='TNC', jac=regularized_gradient)

In [20]:
res

     fun: 19019.77458025878
     jac: array([ 6.54045172, -2.55516279, -3.49690629, ...,  1.89485705,
        4.65025859, -0.14947923])
 message: 'Converged (|f_n-f_(n-1)| ~= 0)'
    nfev: 304
     nit: 17
  status: 1
 success: True
       x: array([-0.08112985, -0.19456319,  0.23113082, ...,  0.14653795,
        1.0194764 , -0.10973208])

In [21]:
X_trained, theta_trained = deserialize(res.x, n_movie, n_user, n_feature)
X_trained.shape, theta_trained.shape

((1682, 50), (944, 50))

In [22]:
prediction = X_trained @ theta_trained.T
user0_pred = prediction[:, 0] + Y.mean()
top_idx = np.argsort(user0_pred)[::-1] # descending order
top_idx.shape

(1682,)

In [23]:
user0_pred[top_idx][:10]

array([3.68087768, 3.6409438 , 3.60592812, 3.581428  , 3.43157978,
       3.39773821, 3.39669468, 3.24934632, 3.23905769, 3.21804971])

In [24]:
print('Top recommendations for you:')
movie_list = np.array(movie_list)
for i in top_idx[:10]:
    print('Prediction rating {:.2f} for movie {}'.format(user0_pred[i], movie_list[i]))
# for m in movie_list[top_idx][:10]:
#     print(m)

Top recommendations for you:
Prediction rating 3.68 for movie Star Wars (1977)
Prediction rating 3.64 for movie Titanic (1997)
Prediction rating 3.61 for movie Toy Story (1995)
Prediction rating 3.58 for movie Truth About Cats & Dogs, The (1996)
Prediction rating 3.43 for movie Boot, Das (1981)
Prediction rating 3.40 for movie Return of the Jedi (1983)
Prediction rating 3.40 for movie Princess Bride, The (1987)
Prediction rating 3.25 for movie Empire Strikes Back, The (1980)
Prediction rating 3.24 for movie Close Shave, A (1995)
Prediction rating 3.22 for movie Wrong Trousers, The (1993)
